## Importo le librerie

In [1]:
import pandas as pd
import re

## Importo i dataset

In [2]:
Esselunga = pd.read_csv('https://raw.githubusercontent.com/CoroTheBoss/DM-project/main/esselunga_preprocessed.csv')
Carrefour = pd.read_csv('https://raw.githubusercontent.com/CoroTheBoss/DM-project/main/carrefour_preprocessed.csv', index_col=0)
Conad = pd.read_csv('https://raw.githubusercontent.com/CoroTheBoss/DM-project/main/conad_preprocessed.csv')

In [51]:
Esselunga.head()

,Category,Product,Price,Quantity
0,spesa bio,"esselunga bio, cipolle dorate 500 g",3.16 € / kg,0.5 kg
1,spesa bio,"esselunga bio, datterini biologici 300 g",8.27 € / kg,0.3 kg
2,spesa bio,"esselunga bio, insalatina 100 g",14.80 € / kg,0.1 kg
3,spesa bio,"esselunga bio, 6 uova fresche biologiche",0.49 € / pz,6.0 pz
4,spesa bio,"esselunga bio, prezzemolo tritato biologico su...",23.00 € / kg,0.05 kg


In [52]:
Conad.head()

,Product,Price,Category,Quantity
0,"Arance Valencia Italia cal. 7 1,5 kg Percorso ...",2.65 € / Kg,Frutta e verdura,1.5 kg
1,Pompelmi Rossi Cal. 4/5 Cat. I,2.48 € / Kg,Frutta e verdura,NaN
2,Limone Costa d'Amalfi I.G.P. Primofiore Italia...,5.56 € / Kg,Frutta e verdura,0.5 kg
3,"Limoni Verdello Italia 0,500 kg Verso Natura C...",3.96 € / Kg,Frutta e verdura,0.5 kg
4,Percorso Qualità Limoni Italia Cal. 4 Cat. I R...,3.96 € / Kg,Frutta e verdura,0.5 kg


In [53]:
Carrefour.head()

,Product,Price,Quantity,Category
0,carrefour correttore a nastro maxi 5x12 mm,1.89 € / pz,1.0 pz,articoli per la casa
1,carrefour confezione di 4 penne a sfera blu pe...,1.49 € / pz,1.0 pz,articoli per la casa
2,carrefour forbici punte arrotondate 17cm,2.99 € / pz,1.0 pz,articoli per la casa
3,carrefour confezione di 4 penne a sfera nero p...,1.49 € / pz,1.0 pz,articoli per la casa
4,carrefour quaderno a4 punto metallico con cope...,1.59 € / pz,1.0 pz,articoli per la casa


# Match

### Librerie

In [6]:
# Library importation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

## Cosine similarity

In [7]:
# Selecting products from datasets
products1 = Carrefour['Product'].tolist()
products2 = Conad['Product'].tolist()
products3 = Esselunga['Product'].tolist()

In [8]:
# Calcolo lunghezze prodotti dei singoli supermercati
len_ca = len(products1)
len_co = len(products2)
len_es = len(products3)

# Creo liste possibili coppie
ca_co = products1 + products2
co_es = products2 + products3
es_ca = products3 + products1

ca_co = [str(x) for x in ca_co]
co_es = [str(x) for x in co_es]
es_ca = [str(x) for x in es_ca]

# Calcolo la matrice delle similarità
vectorizer = TfidfVectorizer()
ca_co_vectors = vectorizer.fit_transform(ca_co)
sim_ca_co = cosine_similarity(ca_co_vectors)
es_ca_vectors = vectorizer.fit_transform(es_ca)
sim_es_ca = cosine_similarity(es_ca_vectors)
co_es_vectors = vectorizer.fit_transform(co_es)
sim_co_es = cosine_similarity(co_es_vectors)

# Defining column order for output
cols = ['Esselunga', 'Carrefour', 'Conad',
        'Esselunga price', 'Carrefour price', 'Conad price', 
        'Esselunga category', 'Carrefour category', 'Conad category',
        'Quantity']

**Carrefour-Conad**

In [15]:
# Blank list for carrefour-conad matches
match_ca_co = []
# Iteration for carrefour products
for i in range(len_ca):
    temp = [] # temporary list
    # Iteration for conad products
    for j in range(len_co):
        # Similarity threshold and price equality condition
        if sim_ca_co[i][len_ca+j]>=0.7 and Carrefour.loc[i, "Quantity"] == Conad.loc[j, "Quantity"]:
            # Appending all match
            temp.append((sim_ca_co[i][len_ca+j],products2[j],j))
    if temp:
        # Selecting the highest similarity match
        prod = max(temp, key=lambda x: x[0])[1]
        index_price = max(temp, key=lambda x: x[0])[2]
        # Appending match pair for final output
        match_ca_co.append((products1[i],prod,
                            Carrefour.loc[i, "Price"],
                            Conad.loc[index_price,"Price"],
                            Carrefour.loc[i, "Category"],
                            Conad.loc[index_price, "Category"],
                            Carrefour.loc[i, "Quantity"]))
    else:
        match_ca_co.append((products1[i],np.nan,
                            Carrefour.loc[i, "Price"],
                            np.nan,
                            Carrefour.loc[i, "Category"],
                            np.nan,
                            Carrefour.loc[i, "Quantity"]))
output_ca_co = pd.DataFrame(match_ca_co, columns=['Carrefour', 'Conad',
                                                  'Carrefour price', 'Conad price',
                                                  'Carrefour category', 'Conad category',
                                                  'Quantity'])

In [16]:
output_ca_co[output_ca_co['Carrefour'].notnull() & output_ca_co['Conad'].notnull()]

,Carrefour,Conad,Carrefour price,Conad price,Carrefour category,Conad category,Quantity
961,carrefour bio latte parzialmente scremato 1 l,Latte Parzialmente Scremato Biologico 1 l Vers...,1.35 € / l,1.79 € / L,prodotti biologici,"Formaggi, latte e uova",1.0 l
966,carrefour bio latte parzialmente scremato micr...,Più Gusto Latte UHT Microfiltrato Parzialmente...,1.75 € / l,1.29 € / L,prodotti biologici,"Formaggi, latte e uova",1.0 l
967,carrefour bio crescenza 100 g,Crescenza 100 g Conad,17.50 € / kg,11.50 € / Kg,prodotti biologici,"Formaggi, latte e uova",0.1 kg
968,sterzing vipiteno bio yogurt da latte fieno bi...,Sterzing Vipiteno bio yogurt da latte fieno bi...,7.67 € / kg,6.60 € / Kg,prodotti biologici,"Formaggi, latte e uova",0.15 kg
973,fiorentini gli originali si&no legumi bio 80 g,Fiorentini gli Originali Si&No Legumi Bio 80 g,29.88 € / kg,23.62 € / Kg,prodotti biologici,Panetteria e snack salati,0.08 kg
...,...,...,...,...,...,...,...
11389,granarolo cucina ad arte mascarpone 250 g,Granarolo Cucina ad Arte Ricotta 250 g,13.96 € / kg,6.76 € / Kg,gastronomia,"Formaggi, latte e uova",0.25 kg
11394,terre d'italia cotechino modena igp 500 g,CONAD Cotechino Modena IGP 500 g,11.98 € / kg,10.58 € / Kg,gastronomia,Carne e salumi,0.5 kg
11400,viva la mamma lasagne al pesto 400 g,Viva la Mamma Lasagne al Ragù 400 g,14.23 € / kg,11.23 € / Kg,gastronomia,Piatti pronti,0.4 kg
11401,riccionepiadina piadina senza glutine 2 x 125 g,Riccionepiadina Piadina Senza Glutine 2 x 125 g,21.56 € / kg,21.8 € / Kg,gastronomia,Preparazioni dolci e salate,0.25 kg


**Carrefour-Esselunga**

In [17]:
match_es_ca = []
for i in range(len_es):
    temp = []
    for j in range(len_ca):
        if sim_es_ca[i][len_es+j]>=0.7 and Esselunga.loc[i, "Quantity"] == Carrefour.loc[j, "Quantity"]:
            temp.append((sim_es_ca[i][len_es+j],products1[j],j))
    if temp:
        prod = max(temp, key=lambda x: x[0])[1]
        index_price = max(temp, key=lambda x: x[0])[2]
        match_es_ca.append((products3[i],prod,
                            Esselunga.loc[i, "Price"],
                            Carrefour.loc[index_price,"Price"],
                            Esselunga.loc[i, "Category"],
                            Carrefour.loc[index_price, "Category"],
                            Esselunga.loc[i, "Quantity"])) 
    else:
        categories = []
        categories.append(Esselunga.loc[i, "Category"])
        match_es_ca.append((products3[i],np.nan,
                            Esselunga.loc[i, "Price"],
                            np.nan,
                            Esselunga.loc[i, "Category"],
                            np.nan,
                            Esselunga.loc[i, "Quantity"])) 
output_es_ca = pd.DataFrame(match_es_ca, columns=['Esselunga', 'Carrefour', 
                                                  'Esselunga price', 'Carrefour price', 
                                                  'Esselunga category', 'Carrefour category',
                                                  'Quantity'])

In [19]:
output_es_ca[output_es_ca['Carrefour'].notnull() & output_es_ca['Esselunga'].notnull()]

,Esselunga,Carrefour,Esselunga price,Carrefour price,Esselunga category,Carrefour category,Quantity
4,"esselunga bio, prezzemolo tritato biologico su...",carrefour bio prezzemolo tritato surgelato 50 g,23.00 € / kg,19.80 € / kg,spesa bio,gelati e surgelati,0.05 kg
5,"esselunga bio, misto per soffritto biologico s...",carrefour bio misto per soffritto surgelato 150 g,7.67 € / kg,6.60 € / kg,spesa bio,gelati e surgelati,0.15 kg
6,"esselunga bio, passata di pomodoro biologica 7...",carrefour bio passata di pomodoro 700 g,1.85 € / kg,1.70 € / kg,spesa bio,condimenti e conserve,0.7 kg
7,"esselunga bio, farina tipo 00 di grano tenero ...",carrefour classic farina di grano tenero tipo ...,1.29 € / kg,0.89 € / kg,spesa bio,pasta riso e farina,1.0 kg
9,"esselunga bio, latte microfiltrato biologico p...",carrefour bio latte parzialmente scremato micr...,1.79 € / l,1.75 € / l,spesa bio,prodotti biologici,1.0 l
...,...,...,...,...,...,...,...
14478,"grey, l'acchiappacolore eco 40 fogli",grey l'acchiappacolore eco 40 fogli,0.22 € / pz,0.16 € / pz,cura casa,cura della casa,40.0 pz
14753,"lavanderina, ammorbidente concentrato fiorito ...",lavanderina ammorbidente concentrato fiorito e...,0.14 € / pz,3.00 € / l,cura casa,cura della casa,2.0 l
15243,"neosan, acqua demineralizzata 5 l",carrefour acqua demineralizzata 5 l,0.38 € / l,0.38 € / l,cura casa,cura della casa,5.0 l
15245,"orphea, salvalana foglietti per cassetti e arm...",orphea salvalana foglietti per cassetti e arma...,0.17 € / pz,0.18 € / pz,cura casa,cura della casa,12.0 pz


**Esselunga-Conad**

In [24]:
match_co_es = []
for i in range(len_co):
    temp = []
    for j in range(len_es):
        if sim_co_es[i][len_co+j]>=0.7 and Conad.loc[i, "Quantity"] == Esselunga.loc[j, "Quantity"]:
            temp.append((sim_co_es[i][len_co+j],products3[j],j))
    if temp:
        prod = max(temp, key=lambda x: x[0])[1] 
        index_price = max(temp, key=lambda x: x[0])[2]
        match_co_es.append((products2[i],prod, 
                            Conad.loc[i, "Price"], 
                            Esselunga.loc[index_price,"Price"], 
                            Conad.loc[i, "Category"],
                            Esselunga.loc[index_price, "Category"],
                            Conad.loc[i, "Quantity"]))
    else:
        match_co_es.append((products2[i],np.nan, 
                            Conad.loc[i, "Price"], 
                            np.nan, 
                            Conad.loc[i, "Category"],
                            np.nan,
                            Conad.loc[i, "Quantity"]))      
output_co_es = pd.DataFrame(match_co_es, columns=['Conad', 'Esselunga',
                                                  'Conad price','Esselunga price', 
                                                  'Conad category','Esselunga category',
                                                  'Quantity'])

In [46]:
output_co_es[output_co_es['Conad'].notnull() & output_co_es['Esselunga'].notnull()]

,Conad,Esselunga,Conad price,Esselunga price,Conad category,Esselunga category,Quantity
32,Ananas a Fette 250 g Conad,ananas a fette 250 g,15.9 € / Kg,11.92 € / kg,Frutta e verdura,frutta e verdura,0.25 kg
63,Prugne Denocciolate 250 g Conad,"esselunga, prugne secche denocciolate 250 g",13.00 € / Kg,13.96 € / kg,Frutta e verdura,frutta e verdura,0.25 kg
64,Prugne Denocciolate 500 g Conad,"life, prugne denocciolate 500 g",14.50 € / Kg,15.98 € / kg,Frutta e verdura,frutta e verdura,0.5 kg
65,Uva Sultanina 250 g Conad,"noberasco bio, uva sultanina biologiche 250 g",6.20 € / Kg,11.96 € / kg,Frutta e verdura,spesa bio,0.25 kg
78,Noberasco Viva i Datteri con Nocciolo 200 g,"noberasco, viva i datteri 200 g",24.95 € / Kg,22.95 € / kg,Frutta e verdura,frutta e verdura,0.2 kg
...,...,...,...,...,...,...,...
8027,"Thè alla Pesca 1,5 l Conad","esselunga, thè alla pesca 1,5 l",0.59 € / L,0.72 € / l,Bevande e preparati,"acqua, birra e bibite",1.5 l
8096,Matt Benessere Vigore Vitamine & Minerali 20 c...,"matt, benessere ricarica 20 compresse efferves...",52.22 € / Kg,0.23 € / pz,Prodotti dietetici e integratori,"patatine, snack e integratori",0.09 kg
8100,Enervit Protein 32% Bar with Vitamins Lemon Ca...,"enervit, protein 32% bar with vitamins lemon c...",35.21 € / Kg,41.46 € / kg,Prodotti dietetici e integratori,"patatine, snack e integratori",0.048 kg
8102,Enervit The Protein Deal Choco & Vanilla Dream...,"enervit, the protein deal choco & vanilla drea...",38.00 € / Kg,43.64 € / kg,Prodotti dietetici e integratori,"patatine, snack e integratori",0.055 kg


**Merging**

- conad/carrefour e esselunga/carrefour

In [32]:
final = output_ca_co.copy()
# Iteration over the first dataset
for i in range(len(output_ca_co)):
    # Selecting product
    elem = output_ca_co["Carrefour"][i]
    # Search for product in the second dataset
    if elem in output_es_ca["Carrefour"].tolist():
        # Append product and price
        index = output_es_ca["Carrefour"].tolist().index(elem)
        final.loc[i, "Esselunga"] = output_es_ca.loc[index, "Esselunga"]
        final.loc[i, "Esselunga price"] = output_es_ca.loc[index, "Esselunga price"]
        final.loc[i, "Esselunga category"] = output_es_ca.loc[index, "Esselunga category"]    
    else:
        # Append NaN if there is no match
        final.loc[i, "Esselunga"] = np.nan
        final.loc[i, "Esselunga price"] = np.nan
        final.loc[i, "Esselunga category"] = np.nan
final = final[cols]
final[final['Conad'].notnull() & final['Esselunga'].notnull() & final['Carrefour'].notnull()]

,Esselunga,Carrefour,Conad,Esselunga price,Carrefour price,Conad price,Esselunga category,Carrefour category,Conad category,Quantity
966,"esselunga bio, latte microfiltrato biologico p...",carrefour bio latte parzialmente scremato micr...,Più Gusto Latte UHT Microfiltrato Parzialmente...,1.79 € / l,1.75 € / l,1.29 € / L,spesa bio,prodotti biologici,"Formaggi, latte e uova",1.0 l
967,"esselunga bio, crescenza biologica 100 g",carrefour bio crescenza 100 g,Crescenza 100 g Conad,18.90 € / kg,17.50 € / kg,11.50 € / Kg,spesa bio,prodotti biologici,"Formaggi, latte e uova",0.1 kg
968,"sterzing vipiteno, bio yogurt bianco 150 g",sterzing vipiteno bio yogurt da latte fieno bi...,Sterzing Vipiteno bio yogurt da latte fieno bi...,7.21 € / kg,7.67 € / kg,6.60 € / Kg,spesa bio,prodotti biologici,"Formaggi, latte e uova",0.15 kg
976,"esselunga bio, fagiolini biologici surgelati 4...",carrefour bio fagiolini finissimi surgelati bi...,Fagiolini Finissimi Biologici Surgelati 450 g ...,4.65 € / kg,4.87 € / kg,4.64 € / Kg,spesa bio,prodotti biologici,Surgelati e gelati,0.45 kg
983,"fiorentini, bio gallette di grano saraceno 100 g",carrefour bio gallette di grano saraceno 100 g,Gallette di Grano Saraceno Biologiche 100 g Ve...,27.50 € / kg,20.90 € / kg,19.90 € / Kg,spesa bio,prodotti biologici,Panetteria e snack salati,0.1 kg
...,...,...,...,...,...,...,...,...,...,...
11261,"viva la mamma, insalata con tonno 5 cereali le...","viva la mamma insalata con tonno cereali, legu...","Viva la Mamma Insalata con Tonno Cereali, Legu...",16.32 € / kg,18.14 € / kg,14.32 € / Kg,gastronomia e piatti pronti,gastronomia,Piatti pronti,0.22 kg
11271,"giovanni rana, sfogliavelo tortellini con ripi...",giovanni rana sfogliavelo carne 250 g,Giovanni Rana Sfogliavelo Carne 250 g,14.36 € / kg,14.36 € / kg,15.16 € / Kg,gastronomia e piatti pronti,gastronomia,Pasta e riso,0.25 kg
11272,"giovanni rana, rustici tortellini casarecci 250 g",giovanni rana rustici tortellini casarecci 250 g,Giovanni Rana Rustici Tortellini Carne 250 g,14.36 € / kg,14.76 € / kg,13.80 € / Kg,gastronomia e piatti pronti,gastronomia,Pasta e riso,0.25 kg
11302,"buitoni, la sfoglia bontà di semi pasta sfogli...",buitoni la sfoglia bontà di semi pasta sfoglia...,BUITONI la Sfoglia Integrale Pasta Sfoglia Fre...,13.00 € / kg,13.87 € / kg,9.4 € / Kg,"latticini, salumi e formaggi",gastronomia,Preparazioni dolci e salate,0.23 kg


- conad/esselunga e conad/carrefour

In [33]:
final2 = output_co_es.copy()
for i in range(len(output_co_es)):
    elem = output_co_es["Conad"][i]

    if elem in output_ca_co["Conad"].tolist():
        index = output_ca_co["Conad"].tolist().index(elem)
        final2.loc[i, "Carrefour"] = output_ca_co.loc[index, "Carrefour"]
        final2.loc[i, "Carrefour price"] = output_ca_co.loc[index, "Carrefour price"]
        final2.loc[i, "Carrefour category"] = output_ca_co.loc[index, "Carrefour category"]
        
    else:
        final2.loc[i, "Carrefour"] = np.nan
        final2.loc[i, "Carrefour price"] = np.nan
        final2.loc[i, "Carrefour category"] = np.nan

In [34]:
final2 = final2[cols]
final2[final2['Conad'].notnull() & final2['Esselunga'].notnull() & final2['Carrefour'].notnull()]

,Esselunga,Carrefour,Conad,Esselunga price,Carrefour price,Conad price,Esselunga category,Carrefour category,Conad category,Quantity
65,"noberasco bio, uva sultanina biologiche 250 g",carrefour bio uva sultanina biologica 250 g,Uva Sultanina 250 g Conad,11.96 € / kg,10.72 € / kg,6.20 € / Kg,spesa bio,prodotti biologici,Frutta e verdura,0.25 kg
85,"sunsweet, prugne california premium denocciola...",sunsweet prugne california premium denocciolat...,Sunsweet Prugne California premium denocciolat...,17.88 € / kg,27.92 € / kg,27.16 € / Kg,frutta e verdura,frutta e verdura,Frutta e verdura,0.25 kg
86,"sunsweet, prugne california premium denocciola...",sunsweet prugne california premium denocciolat...,Sunsweet Prugne California premium denocciolat...,17.96 € / kg,17.96 € / kg,18.98 € / Kg,frutta e verdura,frutta e verdura,Frutta e verdura,0.5 kg
100,"esselunga, mandorle sgusciate 250 g",carrefour il mercato mandorle sgusciate 250 g,Mandorle Sgusciate 250 g Conad,17.96 € / kg,16.60 € / kg,15.96 € / Kg,frutta e verdura,frutta e verdura,Frutta e verdura,0.25 kg
134,"esselunga, mandorle pelate 40 g",mandorle pelate 40 g,"Noberasco € 0,99 Mandorle Pelate 40 g",24.75 € / kg,24.75 € / kg,24.75 € / Kg,frutta e verdura,frutta e verdura,Frutta e verdura,0.04 kg
...,...,...,...,...,...,...,...,...,...,...
7963,"smart, bicarbonato di sodio purissimo 500 g",carrefour bicarbonato di sodio 500 g,Bicarbonato di sodio purissimo 500 g Conad,1.30 € / kg,1.38 € / kg,2.20 € / Kg,"patatine, snack e integratori",dolci e prima colazione,Bevande e preparati,0.5 kg
8096,"matt, benessere ricarica 20 compresse efferves...",matt benessere vigore vitamine & minerali 20 c...,Matt Benessere Vigore Vitamine & Minerali 20 c...,0.23 € / pz,59.89 € / kg,52.22 € / Kg,"patatine, snack e integratori",cura della persona,Prodotti dietetici e integratori,0.09 kg
8100,"enervit, protein 32% bar with vitamins lemon c...",enervit protein 32% bar with vitamins lemon ca...,Enervit Protein 32% Bar with Vitamins Lemon Ca...,41.46 € / kg,47.71 € / kg,35.21 € / Kg,"patatine, snack e integratori",cura della persona,Prodotti dietetici e integratori,0.048 kg
8102,"enervit, the protein deal choco & vanilla drea...",enervit the protein deal choco & vanilla dream...,Enervit The Protein Deal Choco & Vanilla Dream...,43.64 € / kg,46.36 € / kg,38.00 € / Kg,"patatine, snack e integratori",cura della persona,Prodotti dietetici e integratori,0.055 kg


- carrefour/esselunga e conad/esselunga

In [35]:
final3 = output_es_ca.copy()
for i in range(len(output_es_ca)):
    elem = output_es_ca["Esselunga"][i]
    
    if elem in output_co_es["Esselunga"].tolist():
        index = output_co_es["Esselunga"].tolist().index(elem)
        final3.loc[i, "Conad"] = output_co_es.loc[index, "Conad"]
        final3.loc[i, "Conad price"] = output_co_es.loc[index, "Conad price"]
        final3.loc[i, "Conad category"] = output_co_es.loc[index, "Conad category"]
        
    else:
        final3.loc[i, "Conad"] = np.nan
        final3.loc[i, "Conad price"] = np.nan
        final3.loc[i, "Conad category"] = np.nan

In [36]:
final3 = final3[cols]
final3[final3['Conad'].notnull() & final3['Esselunga'].notnull() & final3['Carrefour'].notnull()]

,Esselunga,Carrefour,Conad,Esselunga price,Carrefour price,Conad price,Esselunga category,Carrefour category,Conad category,Quantity
5,"esselunga bio, misto per soffritto biologico s...",carrefour bio misto per soffritto surgelato 150 g,CONAD Percorso Qualità Misto per Soffritto Sur...,7.67 € / kg,6.60 € / kg,7.67 € / Kg,spesa bio,gelati e surgelati,Surgelati e gelati,0.15 kg
17,"esselunga bio, crescenza biologica 100 g",carrefour bio crescenza 100 g,Crescenza Biologica 100 g Verso Natura Bio Conad,18.90 € / kg,17.50 € / kg,12.90 € / Kg,spesa bio,prodotti biologici,"Formaggi, latte e uova",0.1 kg
20,"esselunga bio, minestrone 12 verdure biologico...",carrefour bio minestrone surgelato 450 g,Minestrone 12 Verdure Biologico Surgelato 450 ...,5.32 € / kg,3.89 € / kg,4.42 € / Kg,spesa bio,prodotti biologici,Surgelati e gelati,0.45 kg
22,"esselunga bio, yogurt intero bianco biologico ...",simpl yogurt bianco intero 500 g,Yogurt Bianco Biologico Intero 500 g Verso Nat...,3.38 € / kg,1.70 € / kg,3.90 € / Kg,spesa bio,uova latte e latticini,"Formaggi, latte e uova",0.5 kg
25,"esselunga bio, fagiolini biologici surgelati 4...",carrefour bio fagiolini finissimi surgelati bi...,Fagiolini Finissimi Biologici Surgelati 450 g ...,4.65 € / kg,4.87 € / kg,4.64 € / Kg,spesa bio,prodotti biologici,Surgelati e gelati,0.45 kg
...,...,...,...,...,...,...,...,...,...,...
14103,"omino bianco, fogli salva i colori 56 pezzi",omino bianco fogli salva i colori 56 pz,Omino Bianco Fogli Salva i Colori 56 pz,0.13 € / pz,0.14 € / pz,5390.00 € / L,cura casa,cura della casa,Articoli per la casa,56.0 pz
14432,"grey, l'acchiappacolore ravviva bianchi 16 fogli",grey l'acchiappacolore ravviva nero 16 fogli,GREY l'Acchiappacolore Eco 16 fogli,0.29 € / pz,0.28 € / pz,0.21 € / conf,cura casa,cura della casa,Articoli per la casa,16.0 pz
14478,"grey, l'acchiappacolore eco 40 fogli",grey l'acchiappacolore eco 40 fogli,GREY l'Acchiappacolore 40 fogli,0.22 € / pz,0.16 € / pz,0.21 € / conf,cura casa,cura della casa,Articoli per la casa,40.0 pz
15245,"orphea, salvalana foglietti per cassetti e arm...",orphea salvalana foglietti per cassetti e arma...,Orphea Salvalana Foglietti per Cassetti e Arma...,0.17 € / pz,0.18 € / pz,2.29 € / conf,cura casa,cura della casa,Articoli per la casa,12.0 pz


**Dataset integrato**

In [54]:
# List of dataframes
pdList = [final, final2, final3] 
# Concat dataframes
df = pd.concat(pdList)
# Dropping duplicates
df = df.drop_duplicates(keep='first')
df.reset_index(drop=True, inplace=True)
df.to_csv("integrated.csv")
df

,Esselunga,Carrefour,Conad,Esselunga price,Carrefour price,Conad price,Esselunga category,Carrefour category,Conad category,Quantity
0,NaN,carrefour correttore a nastro maxi 5x12 mm,NaN,NaN,1.89 € / pz,NaN,NaN,articoli per la casa,NaN,1.0 pz
1,NaN,carrefour confezione di 4 penne a sfera blu pe...,NaN,NaN,1.49 € / pz,NaN,NaN,articoli per la casa,NaN,1.0 pz
2,NaN,carrefour forbici punte arrotondate 17cm,NaN,NaN,2.99 € / pz,NaN,NaN,articoli per la casa,NaN,1.0 pz
3,NaN,carrefour confezione di 4 penne a sfera nero p...,NaN,NaN,1.49 € / pz,NaN,NaN,articoli per la casa,NaN,1.0 pz
4,NaN,carrefour quaderno a4 punto metallico con cope...,NaN,NaN,1.59 € / pz,NaN,NaN,articoli per la casa,NaN,1.0 pz
...,...,...,...,...,...,...,...,...,...,...
31528,vaschetta acciaio l30,NaN,NaN,8.99 € / pz,NaN,NaN,cura casa,NaN,NaN,NaN
31529,portatovaglioli metallo,NaN,NaN,9.99 € / pz,NaN,NaN,cura casa,NaN,NaN,NaN
31530,tovaglia antimacchia jungle 240cm,NaN,NaN,18.90 € / pz,NaN,NaN,cura casa,NaN,NaN,NaN
31531,portafoto rattan 13x18,NaN,NaN,16.90 € / pz,NaN,NaN,cura casa,NaN,NaN,NaN
